<a href="https://colab.research.google.com/github/hussainturii/Advanced-Computer-Vision---Deep-Learning/blob/main/5fNN_fully_regularized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger
from tensorflow import keras
import numpy as np
import math

In [13]:
#from previous experiments, the values with the best accuracy are added.

sweep_config = {
    'method' : 'grid',
    'metric' : {
        'name' : 'val_accuracy',
        'goal' : 'maximize'
    },
    'parameters' : {
        'learning_rate' : {'values': [0.0001]},
        'batch_size' : {'values': [32]},
        'img_size' : {'values': [64]},
        'epochs' : {'values': [5]},
        'hidden_nodes' : {'values': [64]},
        'experiment': {'values': ["batchnorm_only", "dropout_only", "full"]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-project fully regularized")

Create sweep with ID: 9rljol9q
Sweep URL: https://wandb.ai/hussaintoori/5-flowers-project%20fully%20regularized/sweeps/9rljol9q


In [14]:
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        model = keras.Sequential()
        model.add(keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))) #frist flatten layer. linear computations

        if config.experiment == 'batchnorm_only': # model with batch normalization
            model.add(keras.layers.Dense(config.hidden_nodes, use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))

        elif config.experiment == 'dropout_only': # model with drop-out regularization
            model.add(keras.layers.Dense(config.hidden_nodes, activation="relu"))
            model.add(keras.layers.Dropout(0.5))

        elif config.experiment == 'full': # model with all reg techniques combined
            model.add(keras.layers.Dense(config.hidden_nodes,
                                          kernel_regularizer=keras.regularizers.l2(0.01),
                                          use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))
            model.add(keras.layers.Dropout(0.5))

        model.add(keras.layers.Dense(len(CLASS_NAMES), activation="softmax"))

        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]
        if config.experiment == 'full':
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)) #when val_accuracy decreases for 3 epochs cont. it terminates

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [15]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6kzhe3of with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	experiment: batchnorm_only
wandb: 	hidden_nodes: 64
wandb: 	img_size: 64
wandb: 	learning_rate: 0.0001


Epoch 1/5
    104/Unknown 188s 2s/step - accuracy: 0.3657 - loss: 1.4991

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


104/104 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.3661 - loss: 1.4978 - val_accuracy: 0.3838 - val_loss: 1.3856
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.5925 - loss: 1.0820 - val_accuracy: 0.4568 - val_loss: 1.3197
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.6792 - loss: 0.9338 - val_accuracy: 0.4757 - val_loss: 1.2888
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.7577 - loss: 0.8067 - val_accuracy: 0.4405 - val_loss: 1.4056
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.8228 - loss: 0.6899 - val_accuracy: 0.4351 - val_loss: 1.4312


batch/accuracy,▁▁▂▂▂▂▂▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
batch/batch_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇██████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▆▆▅▅▅▂▄▄▄▃▃▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂
epoch/accuracy,▁▄▅▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▅▄▂▁
epoch/val_accuracy,▁▇█▅▅
epoch/val_loss,▆▃▁▇█
batch/accuracy,0.82147


...

batch/accuracy,▁▅▅▅▅▅▅▅▅█▆▆▆▆▆▆▆▅▇▇▆▇▆▆▆▇▇██████▇██████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▆▆▆▆▆▆▄▅▄▄▄▄▄▄▃▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▁▃▃▃▃▃▃▃
epoch/accuracy,▁▄▅██
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▅▃▂▁
epoch/val_accuracy,▁▅▆▅█
epoch/val_loss,█▆▃▃▁
batch/accuracy,0.39975


wandb: Agent Starting Run: gpsfchrk with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	experiment: full
wandb: 	hidden_nodes: 64
wandb: 	img_size: 64
wandb: 	learning_rate: 0.0001


Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.3083 - loss: 3.0934 - val_accuracy: 0.4135 - val_loss: 2.5970
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.4393 - loss: 2.6740 - val_accuracy: 0.4297 - val_loss: 2.5090
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.4754 - loss: 2.5182 - val_accuracy: 0.4514 - val_loss: 2.4918
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.4941 - loss: 2.3548 - val_accuracy: 0.5000 - val_loss: 2.3282
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.5478 - loss: 2.2289 - val_accuracy: 0.4757 - val_loss: 2.3030


batch/accuracy,▁▂▂▂▂▂▂▂▂▂▅▅▅▅▄▆▆▆▆▆▆▆▇▇▇▇▆▇▇▇██████████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▇▇▆▆▆▂▄▄▄▄▄▄▁▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch/accuracy,▁▄▅▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▅▄▂▁
epoch/val_accuracy,▁▂▄█▆
epoch/val_loss,█▆▅▂▁
batch/accuracy,0.54177


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
